# MNIST Training
## Day 3

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
# DataLoader是用于加载数据集的模块
import torchvision
# transforms是数据预处理模块，用于将图像转换为张量，并进行归一化
import torchvision.transforms as transforms
# datasets是用于加载数据集的模块
from torchvision import datasets
# DataLoader是用于加载数据集的模块
from torch.utils.data import DataLoader  # 添加这一行


import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.patheffects as path_effects
import matplotlib.patheffects as path_effects
import os
import torch.utils.data as data


### Q: datasets和Dataloader的区别是什么?
#### datasets 和 DataLoader 的区别
1. datasets（数据集类）
作用：定义和加载数据集，提供对原始数据的访问。
特点：
负责数据获取、下载、存储
应用数据预处理（如 transform）
返回一个数据集对象，包含所有样本
可通过索引访问单个样本：dataset[0] 返回一个样本
"""
train_ds = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
# train_ds 包含所有 60,000 个训练样本
# 可以通过 train_ds[0] 访问第一个样本（图像和标签）
"""

2. DataLoader（数据加载器）
作用：将数据集包装成可迭代的批次，用于训练循环。
特点：
将数据集分成批次（batch）
支持多进程加载（num_workers）
支持打乱顺序（shuffle）
支持内存固定（pin_memory，用于 GPU 加速）
返回一个迭代器，每次迭代返回一个批次的数据
示例：
"""
train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
# train_loader 将 60,000 个样本分成多个批次，每批 128 个样本
# 在训练循环中使用：
for images, labels in train_loader:
    # images: [128, 1, 28, 28] 形状的张量（128个图像）
    # labels: [128] 形状的张量（128个标签）
    # 进行训练...
"""

3. 关系和工作流程
原始数据 → datasets.MNIST → 数据集对象 → DataLoader → 批次迭代器 → 训练循环
         (加载和预处理)    (所有样本)    (分批处理)    (批量数据)
在我们的代码中：
"""
# 步骤1: 创建数据集（包含所有样本）
train_ds = datasets.MNIST(...)  # 60,000 个样本

# 步骤2: 创建数据加载器（将数据集分批）
train_loader = DataLoader(train_ds, batch_size=128, ...)  # 约 469 个批次

# 步骤3: 在训练中使用
for batch_images, batch_labels in train_loader:
    # 每次循环处理 128 个样本
    pass
"""

4. 简单类比
datasets = 图书馆（存放所有书籍）
DataLoader = 借书系统（每次借出几本书，可以按顺序或随机借）
总结：datasets 负责数据的定义和访问，DataLoader 负责将数据组织成批次供训练使用。两者配合使用，datasets 提供数据源，DataLoader 提供高效的批量加载机制

#### Task 0: 使用合适的device(cuda,mps,cpu)

In [6]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device

device(type='mps')

#### Task 1: 进行数据预处理，download并加载MNIST数据集

In [11]:
# 数据预处理
# 将图像转换为张量，并进行归一化
# 0.1307 是 MNIST 数据集的均值，0.3081 是 MNIST 数据集的标准差, 这两个数据是从MNIST数据集的官方网站上获取的，如果之后的数据集没有这些公开数据，则需要重新计算
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# ds是数据集的缩写，train_ds是训练数据集，val_ds是验证数据集
# 从MNIST数据集官网下载数据集，并进行预处理
# 下载的数据集会保存在./data/MNIST文件夹下
# 我们已经有数据集了，所以不需要下载，因此先检查是否存在    
 # 如果数据集不存在，则从MNIST数据集官网下载数据集
train_ds = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
val_ds = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=256, shuffle=False, num_workers=2, pin_memory=True)


#### Task 2: 手动检查训练集的形状

In [39]:
# 手动查看数据集里的内容
print(f"训练集大小: {len(train_ds)}")
print(f"验证集大小: {len(val_ds)}")
print(f"图像形状: {train_ds[0][0].shape}")
print(f"type of the lablel: {type(train_ds[0][1])}")
print(train_ds[0][0][0].shape)

训练集大小: 60000
验证集大小: 10000
图像形状: torch.Size([1, 28, 28])
type of the lablel: <class 'int'>
torch.Size([28, 28])


#### Question 1: train_ds是一个什么样的结构？